In [113]:
catalogue = fits.open("../QSO_cat_fuji_dark_healpix.fits.gz")
selected_targetids = catalogue[1].data["TARGETID"][np.where(catalogue[1].data["Z"] >= 2.0)]
selected_targetids

array([ 39633495605117776,  39633491545030901,  39633493587656875,
        39633493591851198,  39633495605119935,  39633491545031256,
        39633493587657805,  39633491545030702,  39633497601607277,
        39633495605119752,  39633497609994880,  39633497597413664,
        39633497597413949,  39633497597413011,  39633499577125885,
        39633493591852133,  39633499572932649,  39633499572931368,
        39633497601608817,  39633497609996860,  39633495617700085,
        39633495613507218,  39633499572931945,  39633495609312400,
        39633497614190289,  39633497614189735,  39633493591851733,
        39633478723044513,  39633480916668425,  39633483093511065,
        39633487359116079,  39633485245186634,  39633485240993119,
        39633485232604190,  39633485232605672,  39633487371699188,
       616094239675122420,  39633489464658033,  39633493583463661,
        39633489473045106,  39633489464658242,  39633493583465214,
        39633491540837630,  39633491540838097,  39633493583465

In [114]:
sv_files = ["dark/79/7941/coadd-sv1-dark-7941.fits",
            "dark/79/7952/coadd-sv1-dark-7952.fits",
            "dark/79/7953/coadd-sv1-dark-7953.fits",
            "dark/81/8139/coadd-sv1-dark-8139.fits",
            "../sv3/dark/91/9144/coadd-sv3-dark-9144.fits",
           ]

In [115]:
for file in sv_files:
    hdul = fits.open(file)
    try:
        del hdul["EXP_FIBERMAP"]
    except:
        pass
    pos = np.where(np.isin(hdul["fibermap"].data["TARGETID"], catalogue[1].data["TARGETID"]))

    # remove unused rows
    names = ["FIBERMAP", "SCORES", 
             "B_FLUX", "B_IVAR", "B_MASK", "B_RESOLUTION", 
             "R_FLUX", "R_IVAR", "R_MASK", "R_RESOLUTION", 
             "Z_FLUX", "Z_IVAR", "Z_MASK", "Z_RESOLUTION",
            ]
    for name in names:
        hdul[name].data = hdul[name].data[pos]
        if name.startswith("B") or name.startswith("R") or name.startswith("Z"):
            try:
                del hdul[name].header["CHECKSUM"]
                del hdul[name].header["DATASUM"]
            except:
                pass

    # remove unused columns in FIBERMAP
    cols = [
        fits.Column(name="TARGETID",
                    format="K",
                    array= hdul["FIBERMAP"].data["TARGETID"],
                   ),
    ]
    hdu = fits.BinTableHDU.from_columns(cols, name="FIBERMAP")
    hdul["FIBERMAP"] = hdu

    # remove unused columns in SCORES
    cols = [
        fits.Column(name="TARGETID",
                    format="K",
                    array= hdul["FIBERMAP"].data["TARGETID"],
                   ),
        fits.Column(name="TSNR2_LYA_B",
                    format="E",
                    array= hdul["SCORES"].data["TSNR2_LYA_B"],
                   ),
        fits.Column(name="TSNR2_LYA_R",
                    format="E",
                    array= hdul["SCORES"].data["TSNR2_LYA_R"],
                   ),
        fits.Column(name="TSNR2_LYA_Z",
                    format="E",
                    array= hdul["SCORES"].data["TSNR2_LYA_Z"],
                   ),
    ]
    hdu2 = fits.BinTableHDU.from_columns(cols, name="SCORES")
    hdul["SCORES"] = hdu2

    hdul.writeto(file, overwrite=True)